<a href="https://colab.research.google.com/github/JeremiahKamama/Spacy-Analysis/blob/main/analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torchvision

     |████████████████████████████████| 2.3MB 8.3MB/s 
     |████████████████████████████████| 3.3MB 44.9MB/s 
     |████████████████████████████████| 901kB 49.4MB/s 


In [2]:
import transformers
from transformers import DistilBertTokenizer, DistilBertConfig,DistilBertModel

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data,datasets
from torch.utils.data import DataLoader
from torchtext.legacy.data import BucketIterator
from torchtext.datasets import IMDB


import random
import time



In [3]:
model_name = DistilBertModel.from_pretrained('distilbert-base-uncased')
NUM_EPOCHS = 3
LEARNING_RATE = 2
MAX_LENGTH = 512
TRUNCATION = True
BATCH_SIZE = 128
WEIGHT_DECAY = 1e-3
SEED = 42
num_classes = 2
FP_16 = dict(opt_level="01")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
eos_token = tokenizer.sep_token
unk_token = tokenizer.unk_token
pad_token = tokenizer.pad_token
init_token = tokenizer.cls_token



In [5]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

In [6]:
max_length = tokenizer.max_model_input_sizes['distilbert-base-uncased']

print(max_length)


512


In [7]:
def tokenize_cut(review):
  tokens = tokenizer.tokenize(review)
  tokens = tokens[:max_length-2]
  return tokens

In [8]:
review = data.Field(batch_first = True,
                    use_vocab = False,
                    tokenize = tokenize_cut,
                    preprocessing = tokenizer.convert_tokens_to_ids,
                    init_token = init_token_idx,
                    eos_token = eos_token_idx,
                    pad_token = pad_token_idx,
                    unk_token = unk_token_idx)

sentiment = data.LabelField(dtype = torch.float)

In [9]:
train_set,test_set = datasets.IMDB.splits(review,sentiment)

df_train, df_val = train_set.split(random_state = random.seed(SEED))


train_iterator,test_iterator = IMDB(split=('train','test'))

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 79.2MB/s]
aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 83.9MB/s]


In [10]:
sentiment.build_vocab(train_set)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_set, test_set), 
    batch_size = BATCH_SIZE, 
    device = device)

In [13]:
class DistilBert(nn.Module):

    def __init__(self, distilbert =None):

        super(DistilBert,self).__init__()

        config = DistilBertConfig.from_pretrained("distilbert-base-uncased")

        self.distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, num_classes)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

    def forward(self, features, attention_mask= None, head_mask=None):

        distilbert_output = self.distilbert(input_ids=features,
                                            attention_mask=attention_mask)
        hidden_state = distilbert_output[0]  # [BATCH_SIZE=32, MAX_SEQ_LENGTH = 512, DIM = 768]
        pooled_output = hidden_state[:, 0]  # [32, 768]
        pooled_output = self.pre_classifier(pooled_output)  # [32, 768]
        pooled_output = F.relu(pooled_output)  # [32, 768]
        pooled_output = self.dropout(pooled_output)  # [32, 768]
        logits = self.classifier(pooled_output)  # [32, 2]

        return logits


In [14]:
model = DistilBert()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [16]:
model = model.to(device)
criterion = criterion.to(device)

In [17]:
def binary_accuracy(preds,y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum()/ len(correct)

    return acc

In [18]:
def train(model,train_iterator,optimizer,criterion):
    model.train()
    for batch in train_iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
    
    #epoch_loss = 0
    #epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.sentiment)
            
            acc = binary_accuracy(predictions, batch.sentiment)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
#trained = train(model,train_iterator,optimizer,criterion)

In [1]:
for epoch in range(NUM_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer,criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

NameError: ignored

In [ ]:
def prediction(model, test_set: int, max_len: int = 512, device = 'cpu'):
    x_encoded = tokenizer.encode_plus(sentence, add_special_tokens=True, pad_to_max_length=True, max_length=max_len, return_tensors="pt",).to(device)
    logits = model(x_encoded['input_ids'], x_encoded['attention_mask'])
    probabilities = F.softmax(logits.detach(), dim=1)
    output = probabilities.max(axis=1)
    print(sentence)
    print(f"Class: {['Negative' if output.indices[0] == 0 else 'Positive'][0]}") 
    #Probability: {output.values[0]:.4f}")

In [ ]:
prediction(model, test_set['sentiment'].values[1])
